In [8]:
import os
import socket

def assert_cpus_disabled(start, end):
    with open('/sys/devices/system/cpu/online', 'r') as f:
        online_cpus = f.read().strip()
        for cpu in range(start, end+1):
            assert str(cpu) not in online_cpus, f"CPU {cpu} is enabled"
def check_cpu_governor(start, end):
    for cpu in range(start, end+1):
        governor_file = f"/sys/devices/system/cpu/cpu{cpu}/cpufreq/scaling_governor"
        if os.path.exists(governor_file):
            with open(governor_file, 'r') as f:
                governor = f.read().strip()
                assert governor == 'performance', f"CPU {cpu} governor is not set to performance"
        else:
            print(f"CPU {cpu} does not exist or does not have a scaling governor")
def find_consecutive_free_ports(start_port, end_port, consecutive_ports):
    for port in range(start_port, end_port):
        for i in range(consecutive_ports):
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
                result = sock.connect_ex(('localhost', port + i))
                if result == 0:
                    break
                if i == consecutive_ports - 1:
                    return port
    return None

# run ~/cpu.sh
check_cpu_governor(0, 15)
assert_cpus_disabled(16, 31)

# Setup evaluation suite

yusung_set = [
    "bullet",
    "factorial",
    "ffmpeg",
    "fractals",
    "funky-kart",
    "game-of-life",
    "gotemplate",
    "hnset-bench",
    'hydro',
    "jqkungfu",
    "lichess",
    "mandelbrot",
    "ogv",
    "onnxjs",
    "pacalc",
    'parquet',
    "playnox",
    "roslyn",
    "rustpython",
    "sandspiel",
    "sqlgui",
    "sqlpractice",
    "takahirox",
    "tic-tac-toe", # flaky
    "timestretch",
    "vaporboy",
    "video",
    "waforth",
    "wasmsh",
    "wheel",
]

jakob_set = [
    "boa",
    "commanderkeen",
    "ffmpeg",
    "fib",
    "figma-startpage",
    "funky-kart",
    "game-of-life",
    "guiicons",
    "handy-tools",
    'heatmap',
    'image-convolute',
    "jsc",
    "kittygame",
    'multiplyDouble',
    "multiplyInt",
    "ogv",
    "pathfinding",
    "riconpacker",
    "rtexviewer",
    "sandspiel",
    "sqlgui",
    "video",
    'visual6502remix',
]

other_set = [
    'livesplit',
    'noisereduction',
    'rfxgen',
    'rguilayout',
    'rguistyler',
    'rtexpacker',
    'skeletal',
    'uarm',
    'vim-wasm',
    'virtualkc',
]

union = list(set(yusung_set) | set(jakob_set) | set(other_set))
print('yusung_set: ', len(yusung_set))
print('jakob_set: ', len(jakob_set))
print('other: ', len(other_set))
print('union: ', len(union))

testset = union
metrics = {testname: { 'trace_match': {}, 'record_metrics': {}, 'replay_metrics': {}} for testname in testset }
start_port = find_consecutive_free_ports(8080, 65535, len(testset))
r3_path = os.getenv('WASMR3_PATH', '~/wasm-r3')

yusung_set:  30
jakob_set:  23
other:  10
union:  56


In [11]:
import subprocess
import json
import concurrent.futures

# Setup artifacts
# frontends = ['custom', 'wasabi', 'firefox', 'webkit']
frontends = ['wasabi']
frontend_to_option = {
    'wasabi': '',
    'firefox': '-f',
    'webkit': '-w',
    'custom': '-c',
}
# parrallel - 6min with 32 pass
# serial - 35min with 34 pass
parallel = False
timeout = 300 if parallel else 120

def run_command(testname, frontend, i):
    frontend_option = frontend_to_option[frontend]
    port = start_port + i
    command = f". ~/.bashrc && timeout {timeout}s npm test -- {frontend_option} -t {testname} -p {port}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    isNormal = result.returncode == 0
    if not isNormal:
        print(result.args)
        print(result.stderr)
    return [testname, frontend, isNormal]

if parallel:
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        futures = [executor.submit(run_command, testname, frontend, i) for i, testname in enumerate(testset) for frontend in frontends]
        results = [future.result() for future in concurrent.futures.as_completed(futures)]
else:
    results = [run_command(testname, frontend, i) for i, testname in enumerate(testset) for frontend in frontends]
for testname, frontend, isNormal in results:
    metrics[testname]['trace_match'][frontend] = isNormal

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

. ~/.bashrc && timeout 300s npm test --  -t fractals -p 8083

. ~/.bashrc && timeout 300s npm test --  -t visual6502remix -p 8081

. ~/.bashrc && timeout 300s npm test --  -t noisereduction -p 8082

. ~/.bashrc && timeout 300s npm test --  -t hnset-bench -p 8087

. ~/.bashrc && timeout 300s npm test --  -t onnxjs -p 8089

. ~/.bashrc && timeout 300s npm test --  -t timestretch -p 8106
node:internal/process/promises:289
            triggerUncaughtException(err, true /* fromPromise */);
            ^

route.fetch: unable to verify the first certificate
Call log:
  - → GET https://superpowered.com/js-wasm-sdk/example_timestretching/
  -   user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/123.0.6312.4 Safari/537.36
  -   accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
  -   accept-encoding: gzip,deflate,br
  -   upgrade-insecure-requests: 1
  -   s

In [35]:
import subprocess
import json
import json
with open('metrics.json', 'r') as f:
    metrics = json.load(f)

timeout = 120 # seconds
engine_kind = ['sm', 'sm-base', 'sm-opt', 'v8', 'v8-liftoff', 'v8-turbofan', 'jsc', 'jsc-int','jsc-bbq','jsc-omg', 'wizeng','wizeng-int','wizeng-jit','wizeng-dyn','wasmtime','wasmer','wasmer-base']
wizard_engine_kind = ['wizeng','wizeng-int','wizeng-jit','wizeng-dyn']
opt_kind = ['noopt', 'split', 'merge', 'custom', 'benchmark'] # custom and benchmark are technically not replay opt though
parallel = False

def get_replay_wasm(testname, opt):
    regex = ''
    match opt:
        case 'noopt':
            regex = 'merge|split|custom|benchmark'
        case 'split':
            regex = 'noopt|merge|custom|benchmark'
        case 'merge':
            regex = 'noopt|split|custom|benchmark'
        case 'custom':
            regex = 'noopt|split|merge|benchmark'
        case 'benchmark':
            regex = 'noopt|split|merge|custom'
        case _:
            exit('invalid op')
    find_command = f"find {r3_path}/tests/online/{testname} -name replay.wasm | grep -vE '{regex}'"
    find_result = subprocess.run(find_command, shell=True, capture_output=True, text=True)
    replay_path = find_result.stdout.strip()
    return replay_path

def run_wish_you_were_fast(testname, engine, opt):
    global metrics
    replay_path = get_replay_wasm(testname, opt)
    command = f". ~/.bashrc && RUNS=1 ENGINES={engine} timeout {timeout}s compare-engines.bash {replay_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    isNormal = 'o' if result.returncode == 0  else ''
    if not isNormal:
        print(result.args)
        print(result.stderr)
        metrics[testname]['replay_metrics'][engine][opt] = {}
    else:
        runtime = float(result.stdout.split(":")[-1].strip())
        metrics[testname]['replay_metrics'][engine][opt] |= { 'runtime': runtime }

def run_wizard(testname, engine, opt):
    global metrics
    replay_path = get_replay_wasm(testname, opt)
    command = f". ~/.bashrc && timeout {timeout}s  wizeng.x86-64-linux --metrics --monitors=profile {replay_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    isNormal = result.returncode == 0
    if not isNormal:
        print(result.args)
        print(result.stderr)
        metrics[testname]['replay_metrics'][engine][opt] = {}
    else:
        monitor, profile = result.stdout.split("pregen:time_us")
        profile = 'pregen:time_us' + profile
        # Make replay_metrics after "pregen:time_us" a key of some object
        metrics[testname]['replay_metrics'][engine][opt] |= {line.rsplit(":", 1)[0].strip(): line.rsplit(":", 1)[1].strip().replace("μs", "").strip() for line in profile.split("\n") if line}

for testname in metrics:
    isNormal = metrics[testname]['trace_match']['wasabi']
    if isNormal:
        for engine in engine_kind:
            metrics[testname]['replay_metrics'][engine] = {}
            for opt in opt_kind:
                metrics[testname]['replay_metrics'][engine][opt] = {}
        for engine in engine_kind:
            for opt in ['benchmark']:
                run_wish_you_were_fast(testname, engine, opt)
        # for engine in wizard_engine_kind:
        for engine in ['wizeng-int']:
            for opt in opt_kind:
                run_wizard(testname, engine, opt)

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

. ~/.bashrc && timeout 60s  wizeng.x86-64-linux --metrics --monitors=profile 

. ~/.bashrc && RUNS=1 ENGINES=wizeng-jit timeout 60s compare-engines.bash /home/don/wasm-r3/tests/online/mandelbrot/benchmark/bin_0/replay.wasm

. ~/.bashrc && RUNS=1 ENGINES=wizeng-dyn timeout 60s compare-engines.bash /home/don/wasm-r3/tests/online/mandelbrot/benchmark/bin_0/replay.wasm

. ~/.bashrc && timeout 60s  wizeng.x86-64-linux --metrics --monitors=profile 

. ~/.bashrc && timeout 60s  wizeng.x86-64-linux --metrics --monitors=profile 

. ~/.bashrc && RUNS=1 ENGINES=wizeng-jit timeout 60s compare-engines.bash /home/don/wasm-r3/tests/online/rfxgen/benchmark/bin_0/replay.wasm
!NullCheckException
	in X86_64Assembler.emit_rex_bb_r_m() [/home/don/titzer/virgil/lib//asm/x86-64/X86_64Assembler.v3 @ 2095:38]
	in X86_64Assembler.movss_m_s() [/home/don/titzer/virgil/lib//asm/x86-64/X86_64Assembler.v3 @ 877:32]
	in X86_64MacroAssembler.emit_mov_m_r() [src/engine/x86-64/X86_64MacroAssembler.v3 @ 260:45]
	in Macro

In [37]:
import json
from tabulate import tabulate
with open('metrics.json', 'r') as f:
    metrics = json.load(f)

print('RQ1: Applicability')

print('5.2.1 Record Experiment')
rq1_results = [['Test Name', 'Trace Match']] + sorted([[testname, 'o' if metrics[testname]['trace_match']['wasabi'] else ''] for testname in metrics]) + [['Total', sum([1 for testname in metrics if metrics[testname]['trace_match']['wasabi']])]]
print(tabulate(rq1_results, tablefmt="latex"))

print('5.2.2 Replay Experiment')
def get_engine_runtime(testname):
    return [metrics[testname]['replay_metrics'][engine]['benchmark'].get('runtime') for engine in engine_kind]
rq22_results = [['Test name', *engine_kind]] + sorted([[testname, *get_engine_runtime(testname)] for testname in metrics if metrics[testname]['trace_match']['wasabi'] ])
print(tabulate(rq22_results, tablefmt="latex"))

RQ1: Applicability
5.2.1 Record Experiment
\begin{tabular}{ll}
\hline
 Test Name       & Trace Match \\
 boa             & o           \\
 bullet          & o           \\
 commanderkeen   & o           \\
 factorial       & o           \\
 ffmpeg          & o           \\
 fib             & o           \\
 figma-startpage & o           \\
 fractals        &             \\
 funky-kart      & o           \\
 game-of-life    & o           \\
 gotemplate      &             \\
 guiicons        & o           \\
 handy-tools     & o           \\
 heatmap         &             \\
 hnset-bench     &             \\
 hydro           & o           \\
 image-convolute &             \\
 jqkungfu        & o           \\
 jsc             & o           \\
 kittygame       & o           \\
 lichess         &             \\
 livesplit       &             \\
 mandelbrot      & o           \\
 multiplyDouble  & o           \\
 multiplyInt     & o           \\
 noisereduction  &             \\
 ogv        

In [28]:
import json
from tabulate import tabulate
with open('metrics.json', 'r') as f:
    metrics = json.load(f)
metrics_with_results = { testname: metrics[testname] for testname in metrics if metrics[testname]['test_success'] }

print('RQ2-1: Performance-Record')
print('TODO')

print('RQ2-2: Performance-Replay')
def get_metric(testname, metric):
    return metrics[testname]['replay_metrics']['wizeng-int']['custom'].get(metric)
rq22_results = [['Test name', 'replay proportion', 'pregen time', 'load time', 'validate time', 'spc time', 'start time', 'main time']] + [[testname, 'TODO', get_metric(testname, 'pregen:time_us'), get_metric(testname, 'load:time_us'), get_metric(testname, 'validate:time_us'), get_metric(testname, 'spc:time_us'), get_metric(testname, 'start:time_us'), get_metric(testname, 'main:time_us')] for testname in metrics_with_results ]
print(tabulate(rq22_results, tablefmt="latex"))

KeyError: 'test_success'

In [56]:
import json
from tabulate import tabulate
with open('metrics.json', 'r') as f:
    metrics = json.load(f)

print('RQ3: Trace Reduction')

print('TODO')

RQ3: Trace Reduction
TODO


In [57]:
import json
from tabulate import tabulate
with open('metrics.json', 'r') as f:
    metrics = json.load(f)
metrics_with_results = { testname: metrics[testname] for testname in metrics if metrics[testname]['test_success'] }

print('RQ4: Replay Optimization')

def get_metric(testname, opt, time):
    metric = metrics[testname]['replay_metrics']['wizeng-int'][opt]
    if len(metric) == 0:
        return 0
    else:
        return metric[time]

print('RQ4-1: Load time')
time = 'load:time_us'
rq4_results = [['Test name', 'noopt time', 'split time', 'merge time', 'fullopt time']] + [[testname, get_metric(testname, 'noopt', time), get_metric(testname, 'split', time), get_metric(testname, 'merge', time), get_metric(testname, 'custom', time)] for testname in metrics_with_results]
print(tabulate(rq4_results, tablefmt="latex"))

print('RQ4-2: Validate time')
time = 'validate:time_us'
rq4_results = [['Test name', 'noopt time', 'split time', 'merge time', 'fullopt time']] + [[testname, get_metric(testname, 'noopt', time), get_metric(testname, 'split', time), get_metric(testname, 'merge', time), get_metric(testname, 'custom', time)] for testname in metrics_with_results]
print(tabulate(rq4_results, tablefmt="latex"))

print('RQ4-3: Main time')
time = 'main:time_us'
rq4_results = [['Test name', 'noopt time', 'split time', 'merge time', 'fullopt time']] + [[testname, get_metric(testname, 'noopt', time), get_metric(testname, 'split', time), get_metric(testname, 'merge', time), get_metric(testname, 'custom', time)] for testname in metrics_with_results]
print(tabulate(rq4_results, tablefmt="latex"))

RQ4: Replay Optimization
RQ4-1: Load time
\begin{tabular}{lllll}
\hline
 Test name    & noopt time & split time & merge time & fullopt time \\
 bullet       & 12082      & 12073      & 11757      & 11666        \\
 factorial    & 772        & 767        & 769        & 769          \\
 fractals     & 38         & 36         & 38         & 37           \\
 funky-kart   & 0          & 131955     & 14681      & 13241        \\
 game-of-life & 57         & 55         & 52         & 53           \\
 hnset-bench  & 35         & 36         & 35         & 35           \\
 mandelbrot   & 13850      & 13728      & 1487       & 1492         \\
 ogv          & 30         & 28         & 29         & 27           \\
 pacalc       & 4763       & 4752       & 4811       & 4647         \\
 sandspiel    & 18002      & 18166      & 17777      & 18175        \\
 sqlgui       & 13468      & 13352      & 14585      & 13391        \\
 sqlpractice  & 39566      & 41296      & 38642      & 24466        \\
 tic-